In [12]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as sp_stats
import scipy.optimize as sp_optimize
from scipy.stats import linregress
from sklearn.neighbors import KernelDensity

# Configuration settings
plt.rcParams.update({
    'figure.figsize': (15, 15),
    'mathtext.fontset': 'cm',
    'axes.titlepad': 20,
    'axes.titlesize': 40,
    'font.size': 40,
    'font.family': 'arial'
})
pd.set_option('display.max_columns', None)

In [13]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy import stats as sp_stats

def rw_i_objective(pars, trials, fb_conditions, df_feedback, responses):
    """
    Objective function for rw_I model.

    :param pars: Unity model parameters.
    :param trials: Array of trial numbers.
    :param fb_conditions: Array of feedback condition per trial.
    :param df_feedback: DataFrame containing feedback data.
    :param responses: Array of response values of one subject.
    :return: Negative log likelihood value.
    """
    a, b, sd = pars
    num_trials = len(trials)
    model = np.zeros(num_trials)
    model[0] = b
    
    for i in range(num_trials - 1):
        current_value = model[i]
        feedback = df_feedback[
            (df_feedback['Estimate'] == np.round(current_value)) & 
            (df_feedback['Condition'] == fb_conditions[i])
        ]['Feedback'].values[0]
        new_value = current_value + a * (feedback - current_value)
        model[i + 1] = max(min(new_value, 100), 0)
    
    return -(sp_stats.norm(model, sd).logpdf(responses).sum())

def rw_ii_objective(pars, trials, fb_conditions, df_feedback, responses):
    """
    Objective function for rw_II model.

    :param pars: Valence model parameters.
    :param trials: Array of trial numbers.
    :param fb_conditions: Array of feedback conditions per trial.
    :param df_feedback: DataFrame containing feedback data.
    :param responses: Array of response values  one subject
    :return: Negative log likelihood value.
    """
    a, b, sd = pars[:3], pars[3], pars[4]
    num_trials = len(trials)
    model = np.zeros(num_trials)
    model[0] = b
    
    for i in range(num_trials - 1):
        current_value = model[i]
        fb_condition = fb_conditions[i - 1]
        feedback = df_feedback[
            (df_feedback['Estimate'] == np.round(current_value)) & 
            (df_feedback['Condition'] == fb_condition)
        ]['Feedback'].values[0]
        alpha = np.sum(a * (feedback_conditions == fb_condition))
        new_value = current_value + alpha * (feedback - current_value)
        model[i + 1] = max(min(new_value, 100), 0)
    
    return -(sp_stats.norm(model, sd).logpdf(responses).sum())

def fit_model(bounds, objective_func, num_params, trials, fb_conditions, df_feedback, responses):
    """
    Fits a model using the specified objective function and parameters.

    :param bounds: Bounds for the parameters.
    :param objective_func: Objective function for the optimization.
    :param num_params: Number of parameters in the model.
    :param trials: Array of trial numbers.
    :param fb_conditions: Array of feedback conditions per trial.
    :param df_feedback: DataFrame containing feedback data.
    :param responses: Array of response values.
    :return: Best fit parameters.
    """
    fit_results = np.zeros((nr_of_starts, num_params))
    for i in range(nr_of_starts):
        start_values = [np.random.uniform(*b) for b in bounds]
        solution = minimize(
            objective_func, start_values, 
            args=(trials, fb_conditions, df_feedback, responses), 
            method='SLSQP', bounds=bounds
        )
        if solution.success:
            fit_results[i] = np.hstack([solution.x, solution.fun])
    
    return fit_results[np.argmin(fit_results[:, -1])]

def calculate_aic_bic(df, num_params, num_trials, func_col):
    """
    Calculates the Akaike Information Criterion (AIC) and Bayesian Information Criterion (BIC).

    :param df: DataFrame containing the model's log-likelihood values.
    :param num_params: Number of parameters in the model.
    :param num_trials: Number of trials in the dataset.
    :param func_col: Column name for the function's log-likelihood value.
    :return: A tuple of AIC and BIC values.
    """
    aic = 2 * num_params + 2 * df[func_col]
    bic = num_params * np.log(num_trials) + 2 * df[func_col]
    return aic, bic

In [14]:
# Load and process data
df = pd.read_csv('../data/variable_fb_data_processed.csv')
df_feedback = pd.read_csv('../data/variable_fb_feedback.csv')
df['pe'] = df['feedback'] - df['confidence']
feedback_conditions = np.array(['pos', 'neut', 'neg'])

# Filter based on prediction error
pe_bound = 1
df_pe_1 = df[np.abs(df['pe']) >= pe_bound]
df_pars = df[['pid', 'bdi_score', 'bdi_group_num', 'bdi_group']].drop_duplicates('pid').reset_index(drop=True)

# Define constants and fit models
bound = (0, 1)
bounds_I = (bound, (1, 99), (0, 30))
bounds_II = (bound, bound, bound, (1, 99), (0, 30))
columns_I = ['alpha_rw_I', 'bias_rw_I', 'noise_rw_I', 'func_rw_I']
columns_II = ['alpha_pos_rw_II', 'alpha_neut_rw_II', 'alpha_neg_rw_II', 'bias_rw_II', 'noise_rw_II', 'func_rw_II']
nr_of_starts = 10

idx = 0
for idx_subj, pid in enumerate(df_pars['pid'].unique()):
    print(f'Subject {idx_subj}')
    subject_data = df_pars[df_pars['pid'] == pid].iloc[0]
    df_pars.loc[idx, ['pid', 'bdi_score', 'bdi_group_num', 'bdi_group']] = subject_data[['pid', 'bdi_score', 'bdi_group_num', 'bdi_group']]
    df_pars.loc[idx, ['pe_bound', 'bound', 'start', 'error_metric']] = ['pe>=1', str(bound), str(bound), 'll']
    responses = df_pe_1[df_pe_1['pid'] == pid]['confidence'].values
    fb_conditions = df_pe_1[df_pe_1['pid'] == pid]['condition'].values
    trials = df_pe_1[df_pe_1['pid'] == pid]['trial'].values
    
    # Fit model
    df_pars.loc[idx, columns_I] = fit_model(bounds_I, rw_i_objective, len(columns_I), trials, fb_conditions, df_feedback, responses)
    df_pars.loc[idx, columns_II] = fit_model(bounds_II, rw_ii_objective, len(columns_II), trials, fb_conditions, df_feedback, responses)
    idx += 1

# Define groups and calculate AIC/BIC
df_pars = define_groups(df_pars, 'bdi_score')
num_trials = 15  # Assuming this is the number of trials

df_pars['aic_I'], df_pars['bic_I'] = calculate_aic_bic(df_pars, 3, num_trials, 'func_rw_I')
df_pars['aic_II'], df_pars['bic_II'] = calculate_aic_bic(df_pars, 5, num_trials, 'func_rw_II')

# Difference in AIC/BIC
df_pars['aic_diff'] = df_pars['aic_II'] - df_pars['aic_I']
df_pars['bic_diff'] = df_pars['bic_II'] - df_pars['bic_I']

Subject 0


/Users/annelene/opt/anaconda3/envs/env-metacog/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:1911: RuntimeWarning: divide by zero encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/annelene/opt/anaconda3/envs/env-metacog/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:1911: RuntimeWarning: divide by zero encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


Subject 1


/Users/annelene/opt/anaconda3/envs/env-metacog/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:1911: RuntimeWarning: divide by zero encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/annelene/opt/anaconda3/envs/env-metacog/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:1911: RuntimeWarning: divide by zero encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


Subject 2


...

Subject 10


/Users/annelene/opt/anaconda3/envs/env-metacog/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:1911: RuntimeWarning: divide by zero encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/annelene/opt/anaconda3/envs/env-metacog/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:1911: RuntimeWarning: divide by zero encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


Subject 11


/Users/annelene/opt/anaconda3/envs/env-metacog/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:1911: RuntimeWarning: divide by zero encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


KeyboardInterrupt: 

In [ ]:
df_pars.to_csv('../results/variable_fb_fitted_model_params.csv')